<a href="https://colab.research.google.com/github/youssefchlendi/ProjetIOT/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet IOT : Détection d'objets en temps réel avec YOLOv4Sticky Keys

## Aperçu du projet
Ce projet démontre la détection d'objets en temps réel en utilisant le modèle YOLOv4. Les objets détectés sont pour l'instant imprimés dans la console, à l'avenir nous enverrons les données à un broker MQTT et les stockerons dans une base de données. Le projet est implémenté en Python et utilise OpenCV, NumPy, et d'autres bibliothèques pour les tâches de vision par ordinateur.

## Étapes à suivre

1. **Importer des bibliothèques** : Importez les bibliothèques nécessaires, notamment OpenCV, NumPy, PIL, etc.
2. **Clonage du dépôt YOLOv4 Dépôt du Darknet**: Clonez le dépôt YOLOv4 depuis GitHub.
3. **Configuration YOLOv4** : Modifier le Makefile pour activer le GPU, OpenCV, et les autres configurations nécessaires.
4. **Compilation de YOLOv4** : Compilez le code de YOLOv4.
5. **Téléchargement des poids pré-entraînés** : Téléchargez les poids pré-entraînés de YOLOv4.
6. **Charge du modèle YOLOv4** : Charger le modèle YOLOv4 avec la configuration et les poids.
7. **Test du modèle YOLOv4** : Testez le modèle YOLOv4 sur une liste d'images.

## Étapes détaillées



Importer des bibliothèques

In [ ]:
# Installations des bibliothèques nécessaires
!pip install paho-mqtt
!pip install pymong

# Importation des bibliothèques nécessaires

from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline

Clonage du dépôt YOLOv4 Dépôt du Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet
# - Activation support OpenCV (OPENCV=1) 
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
# - Activation support GPU (GPU=1) 
!sed -i 's/GPU=0/GPU=1/' Makefile
# - Activation support cuDNN (CUDNN=1) 
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
# - Demi-precision en virgule flottante dans cuDNN (CUDNN_HALF=1) 
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
# - Construction en tant que bibliothèque d'objets partagés (LIBSO=1)
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

 Compilation de YOLOv4

In [ ]:
# Compilation de Darknet
!make

In [ ]:
# Téléchargement des poids pré-entraînés YOLOv4-CSP
!wget \
  --load-cookies /tmp/cookies.txt \
  "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt \
  --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq' \
    -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq" \
      -O yolov4-csp.weights && rm -rf /tmp/cookies.txt

In [ ]:
# Téléchargement des poids pré-entraînés YOLOv4
!wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4.weights -O yolov4.weights

## Exécution du modele


In [ ]:
# Importation des bibliothèques
import cv2
import numpy as np
import json
from google.colab.patches import cv2_imshow  # Import cv2_imshow for displaying images in Colab

# Path de configuration, de données et de poids
cfg_path = "cfg/yolov4.cfg"
data_path = "data/coco.names" 
weights_path = "yolov4.weights"

# Methode pour charger le réseau de neurones
def load_network(cfg_path, data_path, weights_path):
    net = cv2.dnn.readNet(weights_path, cfg_path)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    # Load class names
    with open(data_path, 'r') as f:
        class_names = f.read().strip().split('\n')

    return net, output_layers, class_names

# Charger le réseau de neurones
network, output_layers, class_names = load_network(cfg_path, data_path, weights_path)

# Methode pour détecter les objets
def darknet_helper(img, width, height):
    # Preparation de l'image
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (width, height), swapRB=True, crop=False)
    
    # Charger l'image dans le réseau de neurones
    network.setInput(blob)

    # Exécution de l'inference
    outputs = network.forward(output_layers)

    # Détecter les objets
    detections = []
    # Ce bloc de code est chargé de traiter les sorties du réseau neuronal pour détecter des objets dans une image. Voici un aperçu de ce qu'il fait :

    for output in outputs:

        for detection in output:
            scores = detection[5:]
            # - Obtenir l'identifiant de la classe et le score de confiance de l'objet actuel.
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            # - Vérifier si le score de confiance est supérieur à un seuil (0.5).
            if confidence > 0.5:  
                # - Calculez les coordonnées du centre (x, y), la largeur et la hauteur de la boîte de délimitation.
                center_x = int(detection[0] * img.shape[1])
                center_y = int(detection[1] * img.shape[0])
                
                w = int(detection[2] * img.shape[1])
                h = int(detection[3] * img.shape[0])
                left = int(center_x - w / 2)
                top = int(center_y - h / 2)
                
                # - Ajouter le nom de la classe, le score de confiance et les coordonnées de la boîte de délimitation à la liste des détections.
                detections.append((class_names[class_id], confidence, (left, top, left + w, top + h)))

    return detections

# Exécuter le Védio de Webcam & Envoyer les informations de détection au broker MQTT

In [ ]:
import json
import numpy as np
import cv2
import os
from google.colab.patches import cv2_imshow  
from darknet import *  
import requests
from PIL import Image
from io import BytesIO

# Methode pour obtenir une image aléatoire
def get_random_image():
    url = "https://random.imagecdn.app/1500/1500"
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    # Convertir l'image en RGB (OpenCV utilise BGR)
    return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

# Créer un répertoire pour enregistrer les images avec détections
os.makedirs('/content/images', exist_ok=True)

# Boucle pour afficher 10 images aléatoires avec détections
for count in range(10):
    frame = get_random_image()
    cv2_imshow(frame)  # Afficher l'image
    
    # Créer un tableau de zéros pour stocker les coordonnées des boîtes de délimitation
    bbox_array = np.zeros([frame.shape[0], frame.shape[1], 4], dtype=np.uint8)

    try:
        # Redimensionner l'image à 416x416 pour la détection
        img_resized = cv2.resize(frame, (1500, 1500), interpolation=cv2.INTER_LINEAR)
        
        # Détecter les objets dans l'image
        detections = darknet_helper(img_resized, 1500, 1500)
        print("Detections:", detections)

        # Verifier si des objets ont été détectés
        if detections:
            # Dessiner les boîtes de délimitation sur l'image
            for detection in detections:
                label, confidence, (left, top, right, bottom) = detection
                cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
                cv2.putText(frame, f"{label} [{confidence:.2f}]", (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Enregistrer l'image avec les détections
            image_path = f'/content/images/detected_{count}.jpg'
            cv2.imwrite(image_path, frame)
            print(f"Image saved at {image_path}")

    except Exception as e:
        print("Error during detection:", e)

    # Afficher l'image avec les détections
    cv2_imshow(frame)
